# Necessary imports

In [2]:
import random
import pandas as pd
import networkx as nx
import torch
import torch.nn as nn
import dgl
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from dgl.nn import GraphConv

FILE_PATH = "Download_data_RR.txt"

# Step 1:
Download the RNA-RNA interaction database from the link: http://www.rnainter.org/raidMedia/download/Download_data_RR.tar.gz and extract the file in the current directory.

# Step 2:
Filter the database with Species: Homo sapiens, Category1: miRNA, Category2: mRNA.

In [3]:
data = pd.read_csv(FILE_PATH, sep="\t", dtype=str)

filtered_data = data[(data['Species1'] == 'Homo sapiens') & 
                     (data['Species2'] == 'Homo sapiens') &
                     (data['Category1'] == 'miRNA') & 
                     (data['Category2'] == 'mRNA')]

print(filtered_data.head())


       RNAInterID Interactor1.Symbol Category1      Species1  \
10231  RR00295012      hsa-let-7a-5p     miRNA  Homo sapiens   
10232  RR00297665      hsa-let-7a-5p     miRNA  Homo sapiens   
10233  RR00478746      hsa-let-7a-5p     miRNA  Homo sapiens   
10234  RR00509712      hsa-let-7c-5p     miRNA  Homo sapiens   
10235  RR00518781     hsa-miR-101-3p     miRNA  Homo sapiens   

      Interactor2.Symbol Category2      Species2               Raw_ID1  \
10231              CCND2      mRNA  Homo sapiens  miRBase:MIMAT0000062   
10232               CCR7      mRNA  Homo sapiens  miRBase:MIMAT0000062   
10233               E2F2      mRNA  Homo sapiens  miRBase:MIMAT0000062   
10234              ERCC6      mRNA  Homo sapiens  miRBase:MIMAT0000064   
10235               EZH2      mRNA  Homo sapiens  miRBase:MIMAT0000099   

         Raw_ID2   score                                             strong  \
10231   NCBI:894  0.6905  Western blot//RT-PCR//Reporter assay//qRT-PCR/...   
10232  NCBI:

# Step 3:
Select any 50000 rows from the filtered dataset and make the columns in the order - RNAInter ID,Interactor1,ID1,Category1,Species1,Interactor2,ID2,Category2,Species2,Score,etc.

In [4]:
selected_data = filtered_data.sample(n=50000, random_state=42)

entries = []
for _, row in selected_data.iterrows():
    entry = {
        'RNAInter ID': row['RNAInterID'],
        'Interactor1': row['Interactor1.Symbol'],
        'ID1': row['Raw_ID1'],
        'Category1': row['Category1'],
        'Species1': row['Species1'],
        'Interactor2': row['Interactor2.Symbol'],
        'ID2': row['Raw_ID2'],
        'Category2': row['Category2'],
        'Species2': row['Species2'],
        'Score': row['score']
    }
    entries.append(entry)

print('First 10 entries of the miRNA-mRNA interaction database are as follows ...')
for entry in entries[:10]:
    print(entry)

First 10 entries of the miRNA-mRNA interaction database are as follows ...
{'RNAInter ID': 'RR03762943', 'Interactor1': 'hsa-miR-1238-5p', 'ID1': 'miRBase:MIMAT0022947', 'Category1': 'miRNA', 'Species1': 'Homo sapiens', 'Interactor2': 'NSD1', 'ID2': 'NCBI:64324', 'Category2': 'mRNA', 'Species2': 'Homo sapiens', 'Score': '0.2537'}
{'RNAInter ID': 'RR01692719', 'Interactor1': 'hsa-miR-155-5p', 'ID1': 'miRBase:MIMAT0000646', 'Category1': 'miRNA', 'Species1': 'Homo sapiens', 'Interactor2': 'ADAMTS16', 'ID2': 'NCBI:170690', 'Category2': 'mRNA', 'Species2': 'Homo sapiens', 'Score': '0.2129'}
{'RNAInter ID': 'RR03359131', 'Interactor1': 'hsa-miR-371a-5p', 'ID1': 'miRBase:MIMAT0004687', 'Category1': 'miRNA', 'Species1': 'Homo sapiens', 'Interactor2': 'LCOR', 'ID2': 'NCBI:84458', 'Category2': 'mRNA', 'Species2': 'Homo sapiens', 'Score': '0.2986'}
{'RNAInter ID': 'RR00791142', 'Interactor1': 'hsa-miR-31-5p', 'ID1': 'miRBase:MIMAT0000089', 'Category1': 'miRNA', 'Species1': 'Homo sapiens', 'Intera

# Step 4:
Create a graph out of the dataset with edges as the interacting miRNA and mRNA.

In [5]:
G = nx.Graph()

for entry in entries:
    G.add_edge(entry['ID1'], entry['ID2'], weight=1)

# Select a random node
random_node = random.choice(list(G.nodes))

# Print the adjacency list of the random node
print('Adjacency list of a random node:', random_node)
for neighbor in G.adj[random_node]:
    print(f'Neighbor: {neighbor}, Weight: {G[random_node][neighbor]["weight"]}')

Adjacency list of a random node: NCBI:5310
Neighbor: miRBase:MIMAT0019815, Weight: 1
Neighbor: miRBase:MIMAT0004672, Weight: 1


# Step 5:
Find the largest connected component from the resulting graph.

In [6]:
largest_cc = max(nx.connected_components(G), key=len)
subgraph = G.subgraph(largest_cc)

print('Number of nodes in the largest connected component:', subgraph.number_of_nodes())

# Select a random node in the largest connected component
random_node = random.choice(list(subgraph.nodes))

# Print the adjacency list of the random node in the largest connected component
print('Adjacency list of a random node in the largest connected component:', random_node)
for neighbor in subgraph.adj[random_node]:
    print(f'Neighbor: {neighbor}, Weight: {subgraph[random_node][neighbor]["weight"]}')

Number of nodes in the largest connected component: 16931
Adjacency list of a random node in the largest connected component: miRBase:MIMAT0010133
Neighbor: NCBI:101060445, Weight: 1
Neighbor: NCBI:9337, Weight: 1
Neighbor: NCBI:282996, Weight: 1
Neighbor: NCBI:219988, Weight: 1
Neighbor: NCBI:26084, Weight: 1
Neighbor: NCBI:23196, Weight: 1
Neighbor: NCBI:65108, Weight: 1
Neighbor: NCBI:4800, Weight: 1
Neighbor: NCBI:6045, Weight: 1
Neighbor: NCBI:166614, Weight: 1
Neighbor: NCBI:116496, Weight: 1
Neighbor: NCBI:81608, Weight: 1
Neighbor: NCBI:57649, Weight: 1
Neighbor: NCBI:4249, Weight: 1
Neighbor: NCBI:2272, Weight: 1
Neighbor: NCBI:11135, Weight: 1


# Step 6:
Divide the edges into train and test sets.

In [7]:
# Extract edges from the graph
edges = list(G.edges(data=True))

# Extract features and labels from the edges
edge_features = [(edge[0], edge[1]) for edge in edges]
labels = [edge[2]['weight'] for edge in edges]

train_features, test_features, train_labels, test_labels = train_test_split(edge_features, labels, test_size=0.2, random_state=42)\

print('First 10 train features and labels ...')
for idx in range(10):
    print(train_features[idx], train_labels[idx])

print('First 10 test features and labels ...')
for idx in range(10):
    print(test_features[idx], test_labels[idx])



First 10 train features and labels ...
('miRBase:MIMAT0004564', 'NCBI:26267') 1
('miRBase:MIMAT0004762', 'NCBI:389874') 1
('miRBase:MIMAT0019715', 'NCBI:3746') 1
('miRBase:MIMAT0000449', 'NCBI:140460') 1
('NCBI:481', 'miRBase:MIMAT0014994') 1
('NCBI:64397', 'miRBase:MIMAT0002170') 1
('NCBI:6526', 'miRBase:MIMAT0018935') 1
('NCBI:89795', 'miRBase:MIMAT0018182') 1
('miRBase:MIMAT0018957', 'NCBI:5789') 1
('NCBI:7538', 'miRBase:MIMAT0002806') 1
First 10 test features and labels ...
('miRBase:MIMAT0003225', 'NCBI:148789') 1
('miRBase:MIMAT0003165', 'NCBI:26145') 1
('miRBase:MIMAT0026640', 'NCBI:7849') 1
('miRBase:MIMAT0015086', 'NCBI:6764') 1
('NCBI:1317', 'miRBase:MIMAT0000760') 1
('miRBase:MIMAT0028116', 'NCBI:673') 1
('miRBase:MIMAT0002876', 'NCBI:7021') 1
('NCBI:151647', 'miRBase:MIMAT0019709') 1
('miRBase:MIMAT0004957', 'NCBI:6722') 1
('miRBase:MIMAT0004766', 'NCBI:23554') 1


# Step 7:
Convert the edge list to a DGLGraph.

In [8]:
# Create a dictionary mapping node IDs to their corresponding indices
node_to_index = {node: index for index, node in enumerate(G.nodes)}

# Extract source and destination indices from the edges
src = [node_to_index[edge[0]] for edge in edges]
dst = [node_to_index[edge[1]] for edge in edges]

G_dgl = dgl.graph((src, dst))

# Add self loops to the graph
G_dgl = dgl.add_self_loop(G_dgl)

# Step 8:
Define a simple GNN model for edge prediction

In [9]:
class GNN(nn.Module):
    def __init__(self, in_feats, hidden_feats, num_classes):
        super(GNN, self).__init__()
        self.conv1 = GraphConv(in_feats, hidden_feats)
        self.conv2 = GraphConv(hidden_feats, num_classes)

    def forward(self, g, features):
        x = self.conv1(g, features)
        x = torch.relu(x)
        x = self.conv2(g, x)
        return x

# Step 9:
Define the model and optimizer and prepare the training features and labels.

In [10]:
# Define the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN(in_feats=1, hidden_feats=64, num_classes=1).to(device)

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Prepare features for the nodes (currently only using node IDs)
num_nodes = G_dgl.number_of_nodes()
features = torch.arange(num_nodes).view(-1, 1).float()

# Convert train_labels to PyTorch tensor
train_labels_tensor = torch.tensor(train_labels).float()

# Step 10:
Train any edge prediction algorithm (a basic GNN with randomly initialized nodes).

In [11]:
def train_model(model, g, features, labels, optimizer, num_epochs=1000):
    for epoch in range(num_epochs):
        logits = model(g, features)
        pred = logits.squeeze(1)

        # Select only the labels corresponding to the nodes in the graph
        selected_labels = labels[:g.number_of_nodes()]

        loss = nn.MSELoss()(pred, selected_labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')


# Train the model
train_model(model, G_dgl, features, train_labels_tensor, optimizer)

Epoch 0, Loss: 53032.921875
Epoch 10, Loss: 15047.396484375
Epoch 20, Loss: 1799.99267578125
Epoch 30, Loss: 0.773609459400177
Epoch 40, Loss: 734.9237670898438
Epoch 50, Loss: 527.658203125
Epoch 60, Loss: 10.201738357543945
Epoch 70, Loss: 61.73427963256836
Epoch 80, Loss: 21.539552688598633
Epoch 90, Loss: 1.5870113372802734
Epoch 100, Loss: 0.4328601062297821
Epoch 110, Loss: 0.4879063367843628
Epoch 120, Loss: 0.4191320836544037
Epoch 130, Loss: 0.4044059216976166
Epoch 140, Loss: 0.4211866557598114
Epoch 150, Loss: 0.4172968566417694
Epoch 160, Loss: 0.3978002965450287
Epoch 170, Loss: 0.39099082350730896
Epoch 180, Loss: 0.3887788951396942
Epoch 190, Loss: 0.38479912281036377
Epoch 200, Loss: 0.3818698227405548
Epoch 210, Loss: 0.378610759973526
Epoch 220, Loss: 0.375442236661911
Epoch 230, Loss: 0.37220871448516846
Epoch 240, Loss: 0.3689116835594177
Epoch 250, Loss: 0.3656046986579895
Epoch 260, Loss: 0.36223453283309937
Epoch 270, Loss: 0.3588308095932007
Epoch 280, Loss: 0.3

# Step 11:
Predict and report the results for the nodes in the test set.

In [12]:
with torch.no_grad():
    selected_features = features[:num_nodes]  # Select only the features corresponding to the nodes in the graph
    logits_test = model(G_dgl, selected_features)
    pred_test = logits_test.squeeze(1).numpy()

# Calculate metrics on the test set
mse = nn.MSELoss()(torch.tensor(pred_test[:len(test_labels)]), torch.tensor(test_labels).float())
mae = mean_absolute_error(test_labels, pred_test[:len(test_labels)])
r2 = r2_score(test_labels, pred_test[:len(test_labels)])

print(f"Mean Squared Error on the test set: {mse}")
print(f"Mean Absolute Error on the test set: {mae}")
print(f"R-squared score on the test set: {r2}")

Mean Squared Error on the test set: 0.1782936155796051
Mean Absolute Error on the test set: 0.3832123887033412
R-squared score on the test set: 0.0
